In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Load data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
# batch size
batch_size = 100
# Number of batch 
n_batch = mnist.train.num_examples // batch_size

# Initialize weights
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# Initialize biases
def bias_variablle(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Convolution layer
def conv2d(x,W):
    # x, a 4D tensor: [batch, in_height, in_width, in_channels]
    # W, a filter/kernel shape: [filter_height, filter_width, in_channels, out_channels]
    # strides: Must have `strides[0] = strides[3] = 1`.  For the most common case of the same
    #          horizontal and vertices strides, `strides = [1, stride, stride, 1]`.
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# Pooling layer
def max_pool_2x2(x):
    # ksize [ 1,x,y,1]
    # strides: Must have `strides[0] = strides[3] = 1`.  For the most common case of the same
    #          horizontal and vertices strides, `strides = [1, stride, stride, 1]`.
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# Define placeholder
x = tf.placeholder(tf.float32,[None, 784]) # 第一个数字代表行，784代表有784列
y = tf.placeholder(tf.float32,[None, 10]) # Lable

# change x to 4D: [batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, [-1,28,28,1]) # -1：现在不关心，后续会变成100


W_conv1 = weight_variable([5,5,1,32]) # 5*5*1 kernel size; 32 kernel from 1 plane（一个输入通道）
                                      # 用32个卷积核去采样，最后会得到32个卷积特征平面
b_conv1 = bias_variablle([32]) # 每一个卷积核一个偏置值

# 把x_image和卷积向量进行卷积，再加上偏置，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5,5,32,64]) # 5*5*32 kernel size; 使用64个卷积核对32个平面提取特征；得到64x32个特征平面 (他说是64)??
                                       # 的确是64个，卷的时候是考虑了深度的，卷积核在这里考虑成一个cube(立方体)
b_conv2 = bias_variablle([64]) # 一个卷积核要一个偏置值

# 把h_pool1和卷积向量进行卷积，再加上偏置，然后应用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 28x28的图片第一次卷积后还是28x28(same padding),第一次池化后变14x14(池化窗口2x2)
# 第二次卷积后为14x14,第二次池化后变为7x7
# 通过上面的操作得到64张7x7的平面

# 初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*64,1024]) # 上一层有7x7x64个神经元，定义全连接层有1024个神经元
b_fc1 = bias_variablle([1024]) # 1024个节点

# 把池化层的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64]) # 4D->2D
# 求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32) # How much percentage of node is working
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variablle([10])

# 计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

# 交叉熵代价函数
cross_enropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
# 使用Adam优化器优化
train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(cross_enropy)
# 结果存放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels,keep_prob:1.0})
        print('Iter ' + str(epoch) + ", accuracy = " + str(acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 0, accuracy = 0.1015
Iter 1, accuracy = 0.1021
Iter 2, accuracy = 0.1031
Iter 3, accuracy = 0.1043
Iter 4, accuracy = 0.1053
Iter 5, accuracy = 0.108
Iter 6, accuracy = 0.1105
Iter 7, accuracy = 0.1142
Iter 8, accuracy = 0.117
Iter 9, accuracy = 0.1203
